# **Final LightGBM Model:** Hospital Charge Prediction

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import joblib

In [ ]:
df = pd.read_csv('../data/model_input_final_clean.csv')

In [38]:
# Define feature columns
feature_cols = [
    'gender_encoded', 'age_group_encoded', 'severity_encoded',
    'admission_encoded', 'payment_type_encoded', 'diagnosis_encoded',
    'procedure_encoded', 'county_encoded', 'los',
    'los_x_severity', 'los_x_procedure', 'severity_x_procedure', 'los_x_county',
    'diagnosis_x_severity', 'procedure_x_severity', 
    'diagnosis_x_procedure', 'county_x_los'
]

In [39]:
X = df[feature_cols]
y = np.log1p(df['total_charges'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
base_model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05)
base_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022952 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2544
[LightGBM] [Info] Number of data points in the train set: 1873243, number of used features: 17
[LightGBM] [Info] Start training from score 10.142437


LGBMRegressor(learning_rate=0.05, n_estimators=1000)

In [41]:
# Predict and evaluate on the test set
y_pred_log = base_model.predict(X_test)
y_pred = np.expm1(y_pred_log)  # undo the log1p transformation
y_true = np.expm1(y_test)

# Calculate MAE
mae_global = mean_absolute_error(y_true, y_pred)
print(f"Global Model MAE: ${mae_global:,.2f}")

Global Model MAE: $13,057.92


In [43]:
# Add diagnosis group manually
def categorize_diagnosis(desc):
    desc = desc.lower()
    if any(x in desc for x in ['leukemia', 'lymphoma', 'myeloma', 'neoplasm', 'cancer']):
        return 'Oncology'
    elif any(x in desc for x in ['short gestation', 'liveborn', 'fetal', 'newborn', 'neonatal']):
        return 'Neonatal'
    elif any(x in desc for x in ['schizophrenia', 'mood', 'psychotic', 'substance', 'alcohol']):
        return 'Behavioral'
    elif any(x in desc for x in ['septicemia', 'pneumonia', 'infection']):
        return 'Infectious'
    elif any(x in desc for x in ['osteoarthritis', 'spondylosis', 'back problems']):
        return 'Orthopedic'
    elif any(x in desc for x in ['diabetes', 'heart failure', 'dysrhythmia', 'myocardial', 'coronary']):
        return 'Cardio-Metabolic'
    elif any(x in desc for x in ['pregnancy', 'delivery', 'c-section', 'puerperium']):
        return 'OB/GYN'
    else:
        return 'General'

# Load mapping (you need this .csv)
diag_map = pd.read_csv('../data/diagnosis_mapping.csv')

# Merge readable diagnosis names
df = df.merge(diag_map, on='diagnosis_encoded', how='left')

# Create group
df['diagnosis_group'] = df['ccs_diagnosis_description'].apply(categorize_diagnosis)

In [44]:
from collections import defaultdict
import lightgbm as lgb

group_models = {}
group_maes = {}

# Pull distinct groups
groups = df['diagnosis_group'].dropna().unique()

for group in groups:
    group_df = df[df['diagnosis_group'] == group].copy()

    X_group = group_df[feature_cols]
    y_group = np.log1p(group_df['total_charges'])
    X_train, X_test, y_train, y_test = train_test_split(X_group, y_group, random_state=42)
   
    model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05)
    model.fit(X_train, y_train)

    preds = np.expm1(model.predict(X_test))
    y_true = np.expm1(y_test)
    mae = mean_absolute_error(y_true, preds)

    group_models[group] = model
    group_maes[group] = mae

# Show all group MAEs
for group, mae in group_maes.items():
    print(f"{group} MAE: ${mae:,.2f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009169 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2544
[LightGBM] [Info] Number of data points in the train set: 833469, number of used features: 17
[LightGBM] [Info] Start training from score 10.284842
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001960 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1906
[LightGBM] [Info] Number of data points in the train set: 161873, number of used features: 17
[LightGBM] [Info] Start training from score 10.396232
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001797 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

In [45]:
# Start with global model predictions
df['ensemble_prediction'] = np.expm1(base_model.predict(df[feature_cols]))
df['hybrid_prediction'] = df['ensemble_prediction'].copy()

# Baseline MAEs for comparison
original_group_maes = {
    'Oncology': 19382,
    'Orthopedic': 15300,
    'Cardio-Metabolic': 15100,
    'General': 13700,
    'Infectious': 13300,
    'Behavioral': 7600,
    'Neonatal': 5200,
    'OB/GYN': 4400,
}

# Replace with group predictions only where they improve
for group in df['diagnosis_group'].dropna().unique():
    if group in original_group_maes and group_maes[group] < original_group_maes[group]:
        print(f"✅ Using grouped model for {group} (MAE improved)")
        idx = df['diagnosis_group'] == group
        df.loc[idx, 'hybrid_prediction'] = np.expm1(group_models[group].predict(df.loc[idx, feature_cols]))

In [46]:
final_mae = mean_absolute_error(df['total_charges'], df['hybrid_prediction'])
print(f"\n🏁 Final Hybrid Model MAE: ${final_mae:,.2f}")


🏁 Final Hybrid Model MAE: $13,049.32
